# Imports

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Build fully connected nn

In [2]:
class NN(nn.Module):
    
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    

# model = NN(784, 10)
# print(28*28) 
# x = torch.rand(2, 784)
# print(model(x).shape)

# Set device

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters

In [4]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 1

# Load-data

In [5]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


# initialize a network

In [6]:
model = NN(input_size, num_classes)

# Loss & optimz

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Trian NetWork

In [8]:
for epoch in range(1):
    
    ix = 0 
    
    for images, labels in train_loader:
        # load to device
        images = images.to(device=device)
        labels = labels.to(device=device)
        
        # reshape ---> to 784 --> 28*28
        images = images.reshape(images.shape[0], -1)
        
        # forward 
        scores_pred = model(images)
        # loss & optim
        loss = criterion(scores_pred, labels)
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
        ix += 1


# Accuracy

In [11]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Train data is runing.....')
    else:
        print('Test data is runing.....')
    num_corr = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            x = x.reshape(x.shape[0], -1)
            scores = model(x)
            
            # get max
            _, predictions = scores.max(1)
  
            num_corr += (predictions == y).sum()
            num_samples += predictions.size(0)
        
            
        return float(num_corr) / float(num_samples) * 100
        
    model.train()       

train_acc = check_accuracy(train_loader, model)
test_acc = check_accuracy(test_loader, model)       
print("Train_accu: ", train_acc)
print("Test_accu: ", test_acc)

Train data is runing.....
Test data is runing.....
Train_accu:  93.38
Test_accu:  93.10000000000001
